In [1]:
from slack_data_loader import SlackLoader
import datetime
import pandas as pd
import numpy as np
import re
from operator import itemgetter

In [3]:
path_to_dump = './opendatascience Slack export May 20 2017/'

In [4]:
exporter = SlackLoader(path_to_dump, only_channels=('deep_learning',),
                           start_date=datetime.datetime(2017, 1, 1))

In [5]:
print("Loaded {} messages".format(len(exporter.messages)))

Loaded 7540 messages


In [6]:
channel_attrs = ['id', 'name', 'created', 'creator', 'is_archived', 'is_general', 'pins', 'topic']

def channels_to_df(channels):
    full_list = []
    for ch_id, ch_dict in channels.items():
        new_channel_dict = {}
        for k in channel_attrs:
            new_channel_dict[k] = ch_dict.get(k, None)
        new_channel_dict['num_members'] = len(ch_dict['members'])
        new_channel_dict['purpose'] = ch_dict['purpose']['value']
        full_list.append(new_channel_dict)
    return pd.DataFrame(full_list).set_index('id')


In [7]:
df = channels_to_df(exporter.channels)
df.head()

,created,creator,is_archived,is_general,name,num_members,pins,purpose,topic
id,,,,,,,,,
C2A4GEL6M,1473445368,U04ELQZAU,True,False,alexyashadasha,0,None,,"{'value': '', 'creator': '', 'last_set': '0'}"
C1P8YT7C7,1467817046,U04URBM8V,False,False,bayesian,307,"[{'id': '1467888432.000030', 'type': 'C', 'use...",Church of Bayes: Discussing Bayesian statistic...,"{'value': ':bayes:', 'creator': 'U04ELQZAU', '..."
C0804BS5Q,1437511383,U049NHC4X,False,False,big_data,1301,"[{'id': '1485303977.000947', 'type': 'C', 'use...","Hadoop, Spark и прочее\r\n\r\nПолезные материа...","{'value': 'Big Pain in the ...', 'creator': 'U..."
C0MQQT6E6,1455738772,U070Y25AS,False,False,bioinformatics,125,None,,"{'value': ':bioscience:', 'creator': 'U04ELQZA..."
C115898GZ,1460749144,U04422XJL,True,False,blackoxchallenge,0,None,,"{'value': '', 'creator': '', 'last_set': '0'}"


In [41]:
qwords = ("как", "какой", "зачем", "почему", "когда", "кто", "где", "когда", "куда", "чот")
splitter = re.compile(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s")

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

def lemm(st):
    if st == '':
        return ''
    else:
        return morph.parse(st)[0].normal_form

def is_question(d):
    x = d.lower()
    snt = x.split()
    num_words = len(snt)
    snt = [lemm(w) for w in snt]
    #print((num_words > 4) and  any(w in qwords for w in snt), [w in qwords for w in snt])
    return (num_words > 4) and any(w in qwords for w in snt)

def contains_sentance_with_questions(d):
    x = d['text'].lower()
    sents = splitter.split(x)
    #print(any(map(is_question, sents)))
    return any(map(is_question, sents))

questions = list(filter(contains_sentance_with_questions, exporter.messages))

In [22]:
questions = list(filter(contains_sentance_with_questions, exporter.messages))

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [42]:
print("found {} questions".format(len(questions)))

found 1255 questions


In [43]:
for _text in map(itemgetter('text'), questions):
    print(_text)
    print('-'*40)

1. Variational Inference используется для нахождения апостериорного распределения. А зачем нам нужно апостериорное распределение? Используется ли как-то VI в нейросетях (может это имеет отношение ко второму вопросу)?
----------------------------------------
2. В вероятностном подходе по машинному обучению задача обучения (MAP) ставится как W = argmax(p(D|W) * p(W)), где p(W) - априорное распределение весов, D - обучающая выборка, p(D|W) = p(x_1|W)p(x_2|W)...p(x_N|W), p(x|W) - вероятность появления x, т.е. p(x|W) - не произвольная функция, а положительная и суммируется к единице, Если f(x, W) - нейросеть, то p(x|W)=f(x,W)/integral(f(x, W))dx. Но по факту при обучении никакого интеграла не вычисляется. Как это происходит и где подробнее об этом почитать?
----------------------------------------
<@U0WFHMB4L>: точечная оценка ничего не говорит о том, насколько модель уверена в своём ответе. А вот в байесовском (см. <#C1P8YT7C7|bayesian>) подходе к выводу ищется как раз распределение или ка